In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random

headers = { 
	"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36", 
	"Referer": "https://www.domain.com.au/"
}

base_url = "https://www.domain.com.au/rent/"
errCount = 0
listing_ids = []

for i in range(1,6):
    if i > 1:
        url = f"{base_url}?page={i}"
    else:
        url = base_url
    
    response = requests.get(url, headers=headers)
    if response.ok:
        soup = BeautifulSoup(response.content, 'html')
        elements_with_class = soup.find_all(class_='css-1qp9106')
        
        for element in elements_with_class:
            data_testid_value = element.get('data-testid')
            if data_testid_value:
                listing_id = re.sub(r'\D', '', data_testid_value)
                listing_ids.append(listing_id)

    else:
        print(response)
        errCount += 1


In [3]:
headers = {
    'X-API-Key': 'key_07e6c61e2af44856103c7a58aa5339aa'
    }

base_url = 'https://api.domain.com.au/v1/me'

authenticate = requests.get(base_url, headers=headers)

authenticate.status_code


200

In [4]:
import pandas as pd

base_url = 'https://api.domain.com.au/v1/listings/'
url = f"{base_url}{listing_ids[0]}"
response = requests.get(url, headers=headers)

if response.ok:
    listing = response.json()
else:
    raise RuntimeError

titles = list(listing.keys())

df = pd.DataFrame(columns = titles)


In [5]:
df = df.drop(columns=['geoLocation', 'isNewDevelopment', 'media', 'channel', 
                      'rentalDetails', 'advertiserIdentifiers', 'inspectionDetails', 
                      'objective', 'saleMode', 'description'])

In [13]:
url = f"{base_url}{listing_ids[0]}"

response = requests.get(url, headers=headers)

if response.ok:
    listing = response.json()

listing['priceDetails']['displayPrice']

'$725 per week'

In [6]:
df

,status,id,addressParts,apmIdentifiers,bathrooms,bedrooms,carspaces,dateAvailable,dateUpdated,dateListed,features,headline,priceDetails,propertyId,propertyTypes,seoUrl


In [7]:
for i in listing_ids:
    url = f'{base_url}{i}'
    response = requests.get(url, headers=headers)
    if response.ok:
        listing = response.json()
        df.loc[len(df.index)] = listing

In [8]:
df['addressParts'] = df['addressParts'].apply(lambda x: x['displayAddress'])
df = df.rename(columns={'addressParts': 'address'})

df['apmIdentifiers'] = df['apmIdentifiers'].apply(lambda x: x['suburbId'])
df = df.rename(columns = {'apmIdentifiers': 'suburbId'})

In [9]:
df

,status,id,address,suburbId,bathrooms,bedrooms,carspaces,dateAvailable,dateUpdated,dateListed,features,headline,priceDetails,propertyId,propertyTypes,seoUrl
0,new,17135294,"15/1b Innes Road, Greenwich NSW 2065",19472,1,2,1,2024-07-29T00:00:00+10:00,2024-07-29T06:40:26.323Z,2024-07-29T06:40:26Z,"[Air Conditioning, Close to Transport, Close t...",Tranquil Living with Modern Comforts and Sceni...,"{'canDisplayPrice': False, 'displayPrice': '$7...",SQ-4286-KT,[apartmentUnitFlat],https://www.domain.com.au/15-1b-innes-road-gre...
1,new,17135293,"3b Eskdale Grove, Thrumster NSW 2444",42932,2,3,1,2024-09-02T00:00:00+10:00,2024-07-29T06:40:23.38Z,2024-07-29T06:40:23Z,[Air Conditioning],"Brand New Spacious, Contemporary duplex","{'canDisplayPrice': False, 'displayPrice': '$6...",XA-5948-DK,[semiDetached],https://www.domain.com.au/3b-eskdale-grove-thr...
2,new,17135290,"207/305 Trafalgar Street, Petersham NSW 2049",35402,1,1,1,2024-08-21T00:00:00+10:00,2024-07-29T06:39:59.55Z,2024-07-29T06:39:59Z,NaN,"Spacious, light-filled apartment with designer...","{'canDisplayPrice': False, 'displayPrice': '$7...",SN-3618-CK,[apartmentUnitFlat],https://www.domain.com.au/207-305-trafalgar-st...
3,new,17135276,"12a/30 Wynnstay Rd, Prahran VIC 3181",33397,1,2,0,2024-08-23T00:00:00+10:00,2024-07-29T07:15:55.23Z,2024-07-29T06:36:38Z,NaN,For inspection details contact Katie Love: klo...,"{'price': 520, 'pricePrefix': '$520.00', 'canD...",KH-5577-SJ,[apartmentUnitFlat],https://www.domain.com.au/12a-30-wynnstay-rd-p...
4,new,17135277,"44/10 Homebush Road, Strathfield NSW 2135",40782,1,1,0,2024-08-17T00:00:00+10:00,2024-07-29T06:36:36.78Z,2024-07-29T06:36:36Z,"[City Views, Close to Schools, Close to Shops,...",Self contained studio walk to both Strathfield...,"{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,[apartmentUnitFlat],https://www.domain.com.au/44-10-homebush-road-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,recentlyUpdated,16899117,"8/109-111 Meredith Street, Bankstown NSW 2200",2102,1,2,2,2024-08-05T00:00:00+10:00,2024-07-29T05:16:43.847Z,2024-07-29T05:16:43Z,NaN,Ideal Location,"{'canDisplayPrice': False, 'displayPrice': '$5...",QC-8892-PD,[apartmentUnitFlat],https://www.domain.com.au/8-109-111-meredith-s...
96,recentlyUpdated,14685627,"12/16 Pearl Road, Cloverdale WA 6105",3018,1,2,1,2024-07-31T00:00:00+10:00,2024-07-29T05:15:37.67Z,2024-07-29T05:15:37Z,"[Intercom, Built In Robes, Dishwasher, Air Con...",STYLISH AND SECURE,"{'canDisplayPrice': False, 'displayPrice': '$5...",HM-4873-ZN,[apartmentUnitFlat],https://www.domain.com.au/12-16-pearl-road-clo...
97,new,17134880,"51/180 Maroubra Road, Maroubra NSW 2035",27512,2,3,2,2024-09-03T00:00:00+10:00,2024-07-29T05:14:09.153Z,2024-07-29T05:14:09Z,"[Air Conditioning, Built-In Wardrobes, Close t...",House sized apartment with views,"{'canDisplayPrice': False, 'displayPrice': '$1...",NaN,[apartmentUnitFlat],https://www.domain.com.au/51-180-maroubra-road...
98,new,17134876,"1804/2 Furzer Street, Phillip ACT 2606",1411,1,1,1,2024-07-26T00:00:00+10:00,2024-07-29T07:14:52.827Z,2024-07-29T05:13:45Z,"[Gym, Intercom, Study, Built In Robes, Dishwas...",1 WEEK FREE RENT - WOVA 1-Bedroom Apartment,"{'canDisplayPrice': False, 'displayPrice': '$4...",XF-9405-IN,[apartmentUnitFlat],https://www.domain.com.au/1804-2-furzer-street...


In [42]:
from minio import Minio
# Initialize MinIO client
client = Minio('localhost:9000',
               access_key='accessKey',
               secret_key='secretKey',
               secure=False)
# Make a bucket

df.to_csv(r'./output.csv')

client.fput_object('domain-data', 'output.csv', 'output.csv')


In [10]:
from realestate.common.types_realestate import SearchCoordinate




ModuleNotFoundError: No module named 'realestate'

In [1]:
client = Minio('localhost:9000',
               access_key='accessKey',
               secret_key='secretKey',
               secure=False)

client.list_buckets()

NameError: name 'Minio' is not defined